In [10]:
import os
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Directory containing audio files
audio_dir = "/Testing/"    # replace with your test directory
# Directory to store text files
output_dir = "check"  

# Get list of audio files in the directory
audio_files = os.listdir(audio_dir)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load the pretrained model and processor
processor = WhisperProcessor.from_pretrained("/pretrainedwhisper-medium-native-v3")
model = WhisperForConditionalGeneration.from_pretrained("/pretrainedwhisper-medium-native-v3").to("cuda")

# Set the target sample rate
target_sample_rate = 16000  # Replace with your desired sample rate

# Process each audio file and save transcriptions in text files
for audio_file in audio_files:
    # Load audio file and resample to the target sample rate
    audio_input, _ = librosa.load(os.path.join(audio_dir, audio_file), sr=target_sample_rate, mono=True)

    # Preprocess the audio using the processor
    inputs = processor(audio_input, return_tensors="pt", sampling_rate=target_sample_rate).input_features
    # Perform inference
    with torch.no_grad():
        predicted_ids = model.generate(inputs.to("cuda"))[0]

        transcription = processor.decode(predicted_ids)
        pred = processor.tokenizer._normalize(transcription)


    # Create a text file with the same name as the audio file in the output directoryz
    text_file_name = os.path.splitext(audio_file)[0] + ".txt"
    with open(os.path.join(output_dir, text_file_name), "w", encoding="utf-8") as text_file:
        text_file.write(transcription)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
